In [1]:
#!pip install alive-progress 

In [2]:
from PIL import Image
import json
from PIL import Image ,ImageDraw, ImageFont
from alive_progress import alive_bar; import time

In [3]:
import os

def list_image_paths(folder_path):
    # Get list of all files in the folder
    all_files = os.listdir(folder_path)
    # Filter only image files (assuming common image file extensions)
    image_files = [file for file in all_files if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]
    # Generate full paths for image files
    image_paths = [os.path.join(folder_path, file) for file in image_files]
    
    return image_paths

# # Used for testing list_image_path code function 
# # "image_folder_path"
# image_folder_path = "./annotated_images"
# image_paths = list_image_paths(image_folder_path)

# #Print the list of image paths
# print("Image paths:")
# for path in image_paths:
#     print(path)


In [13]:

def extract_image_file_name(path):
    """
       Given a full path of annotaed image, extract and retrun image file name 
    """
    
    # Split on "/", get 3nd item [2] in list then split again on "." 
    # then file name is first item [0] in new list.
    
    image_file_name =  path.split("/")[2].split(".")[0] 
                                                        
    return image_file_name

In [11]:

def extract_xy_coordinates_mask(path):
    """
       Extract x,y coordinates of the mask in the image and save as JSON
       Inputes: path: path to annotated image 
       Outputs: x, y, coordinates writen to a JSON file saved in JSON folder
       associated with the image name
       
    """
    image = Image.open(path)
    # Convert the image to RGB mode
    image = image.convert("RGB")
    # Get the width and height of the image
    width, height = image.size
    #print(width,height)
    #image.show()
    # List to store x, y coordinates of green pixels
    green_pixels = []
    # Iterate through each pixel of the image
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            r, g, b = image.getpixel((x, y))
            # Define criteria for green (e.g., green if green component is highest)
            if g > r and g > b:
                green_pixels.append((x, y))
    
    # Convert the list of green pixel coordinates to JSON format
    annotations_json = json.dumps({"green_pixels": green_pixels}, indent=4)
    # Save JSON to a file
    with open(JsonFolder + img_name + "_" + "annotations.json", "w") as json_file:
        json_file.write(annotations_json)
    
    #print("Annotations exported as JSON.")

In [12]:

# #########################################################################
# Assumptions:
# #########################################################################
#
# 1. We have annotated images avaiable and saved in annotated images folder
# 2. JSON, Masks, and WhiteMasks folder are created
# 
# Main loop does the following tasks:
#    1. Reads the image
#    2. Generate JSON file with x,y coordinates of the green mask annotations
#    3. Use JSON file to generate new ask only image (green line) same size as orignal image
#    4. ANd finally, Convert the green Mask image to White Mask

# Folders to save outputs of each image processed
#    JSON: x,y coordinates of green mask 
#    Masks: Extracted Green Mask Only
#    WhiteMasks: Generating White Mask

# Initialize output folders 
Masks = "./Masks/"
JsonFolder = "./JSON/"
WhiteMasks = "./WhiteMasks/"

# Initialize path to image input folder
image_folder_path = "./annotated_images"
image_paths = list_image_paths(image_folder_path)

with alive_bar(len(image_paths), force_tty=True, scale="SI", precision=1) as bar:

    for path in image_paths:
        #print(path)
        img_name = extract_image_file_name(path)
        
        # Extract coordinates of mask and save in JSON format
        extract_xy_coordinates_mask (path)        
    
        # ######################################################
        # Generate polygon MASK image
        # Load the JSON file with mask's green pixel coordinates
        # ######################################################
        with open(JsonFolder + img_name + "_" + "annotations.json", "r") as json_file:
            data = json.load(json_file)
        
        # Create a new image with the same dimensions as the original BMP image
        image = Image.open(path)
        width, height = image.size
        mask_image = Image.new("1", (width, height), color=0)  # "1" indicates black and white mode
        
        # Draw the green pixels as white (1) in the mask image
        draw = ImageDraw.Draw(mask_image)
        for x, y in data["green_pixels"]:
            draw.point((x, y), fill=1)
        
        # Save the mask image as PNG
        mask_image.save(Masks + img_name + "_" + "mask.png")
        
        #print(img_name + "_" + "Mask image created and saved as PNG.")

        # ######################################################
        # Generate white MASK image
        # Load the JSON file with green pixel coordinates
        # ######################################################
        with open(JsonFolder + img_name + "_" + "annotations.json", "r") as json_file:
            data = json.load(json_file)
        
        # Create a new image with the same dimensions as the original BMP image
        width, height = image.size
        mask_image = Image.new("1", (width, height), color=0)  # "1" indicates black and white mode
        
        # Draw the green pixels as white (1) in the mask image
        draw = ImageDraw.Draw(mask_image)
        for x, y in data["green_pixels"]:
            draw.point((x, y), fill=1)
        
        # Flood-fill the interior of the mask with white
        ImageDraw.floodfill(mask_image, xy=(0, 0), value=1)
        # Invert the colors so that the inside of the mask becomes white and the outside becomes black
        mask_image = mask_image.point(lambda x: 1 if x == 0 else 0)
        # Save the mask image as PNG
        mask_image.save(WhiteMasks + img_name + "_" + "mask2.png")
        
        #print(img_name + "_" + "Mask image created and saved as PNG.")
    
        # Print the list of green pixel coordinates
        # print("Green Pixels:")
        # for coord in green_pixels:
        #     print(coord)
        bar()


|████████████████████████████████████████| 4/4 [100%] in 5.8s (36.64/m)         


In [17]:


# # Load the BMP image
# image = Image.open(image_path)

# # Convert the image to RGB mode
# image = image.convert("RGB")

# # Get the width and height of the image
# width, height = image.size

# # List to store x, y coordinates of green pixels
# green_pixels = []

# # Iterate through each pixel of the image
# for x in range(width):
#     for y in range(height):
#         # Get the RGB values of the pixel
#         r, g, b = image.getpixel((x, y))
        
#         # Define your criteria for green (e.g., green if green component is highest)
#         if g > r and g > b:
#             green_pixels.append((x, y))

# # Convert the list of green pixel coordinates to JSON format
# annotations_json = json.dumps({"green_pixels": green_pixels}, indent=4)

# # Save JSON to a file
# with open("annotations.json", "w") as json_file:
#     json_file.write(annotations_json)

# print("Annotations exported as JSON.")


In [18]:


# # Load the JSON file containing green pixel coordinates
# with open("annotations.json", "r") as json_file:
#     data = json.load(json_file)

# # Create a new image with the same dimensions as the original BMP image
# width, height = image.size
# mask_image = Image.new("1", (width, height), color=0)  # "1" indicates black and white mode

# # Draw the green pixels as white (1) in the mask image
# draw = ImageDraw.Draw(mask_image)
# for x, y in data["green_pixels"]:
#     draw.point((x, y), fill=1)

# # Save the mask image as PNG
# mask_image.save("mask.png")

# print("Mask image created and saved as PNG.")


In [11]:


# # Load the JSON file containing green pixel coordinates
# with open("annotations.json", "r") as json_file:
#     data = json.load(json_file)

# # Create a new image with the same dimensions as the original BMP image
# width, height = image.size
# mask_image = Image.new("1", (width, height), color=0)  # "1" indicates black and white mode

# # Draw the green pixels as white (1) in the mask image
# draw = ImageDraw.Draw(mask_image)
# for x, y in data["green_pixels"]:
#     draw.point((x, y), fill=1)


# # Flood-fill the interior of the mask with white
# ImageDraw.floodfill(mask_image, xy=(0, 0), value=1)

# # Invert the colors so that the inside of the mask becomes white and the outside becomes black
# mask_image = mask_image.point(lambda x: 1 if x == 0 else 0)

# # Save the mask image as PNG
# mask_image.save("mask2.png")

# print("Mask image created and saved as PNG.")


Mask image created and saved as PNG.


In [64]:
from alive_progress import alive_bar; import time

with alive_bar(1000, force_tty=True) as bar:
    for i in range (1000): 
        time.sleep(0.005)
        if i and i % 300 ==0 :
            print("cool !")
        bar()

on 300: cool !                                                                  
on 600: cool !                                                                  
on 900: cool !                                                                  
|████████████████████████████████████████| 1000/1000 [100%] in 5.2s (193.77/s)  
